# LangChain Crash Course

**Structure of this notebook**:
1. Setup & imports
2. LangChain basics (chains, memory, tools, agents)
3. A small game
4. Upcoming session


## 1. Setup & Imports

In this section we:

- Install all the Python packages we will use later (LangChain core, tools, RAG helpers, widgets).
- Pull in the **core classes** (prompts, chains, memory, tools, agents, document loaders, vector stores).
- Configure the environment once, so the later sections can focus only on ideas and examples.

Run these cells once when you open the notebook. After that, you can jump straight to the basics or the mini‑projects.

In [ ]:
!uv pip install langchain-community pydrive2 faiss-cpu sentence-transformers

Using Python 3.12.12 environment at: /usr
Resolved 106 packages in 1.26s
Prepared 9 packages in 1.27s
Uninstalled 1 package in 3ms
Installed 9 packages in 60ms
 + dataclasses-json==0.6.7
 + faiss-cpu==1.13.1
 + langchain-classic==1.0.0
 + langchain-community==0.4.1
 + langchain-text-splitters==1.0.0
 + marshmallow==3.26.1
 + mypy-extensions==1.1.0
 - requests==2.32.4
 + requests==2.32.5
 + typing-inspect==0.9.0


In [ ]:
!uv pip install pypdf

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 49ms
Prepared 1 package in 30ms
Installed 1 package in 5ms
 + pypdf==6.4.1


In [ ]:
!uv pip install -q langchain-text-splitters

In [ ]:
# Install the necessary packages for the chain
!uv pip install -q langchain-core

In [ ]:
!uv pip install -q langchain-google-genai

In [ ]:
!uv pip install -q duckduckgo-search wikipedia

In [ ]:
!uv pip install -U ddgs

Using Python 3.12.12 environment at: /usr
Resolved 17 packages in 186ms
Prepared 3 packages in 185ms
Installed 3 packages in 19ms
 + ddgs==9.9.3
 + fake-useragent==2.2.0
 + socksio==1.0.0


In [ ]:
# 1. Install the Hub (for pulling the standard agent prompt)
!uv pip install -q langchainhub

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
# Core imports for this notebook

# Standard library
import os
import time

# Optional: Colab Drive (uncomment if using Colab)
# from google.colab import drive

# Display + widgets for simple UI
from IPython.display import display, clear_output
import ipywidgets as widgets

# LangChain core primitives
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory

# LangChain Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI

# LangChain tools & agents
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool
from langchain.agents import create_agent


### Google API key & LLM setup

Make sure you have your `GOOGLE_API_KEY` set securely (e.g., via Colab secrets).

In [ ]:
import os
from google.colab import userdata
# Get API key from Colab secrets manager
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Shared LLM used in all LangChain examples
llm = ChatGoogleGenerativeAI(
    model="gemma-3-27b-it", #"gemini-2.5-flash-lite",
    temperature=0.7,
)


# 2. LangChain: Let's get started


## Pain points
> 1. LLMs are stateless, and can only access current context (input). Context management is crucial.

> 2. The code for intercommunication of Models, Memory, Tools, is a lot of struggle.

## Solutions?


> Let's make a smart context manager, that can help us with prompt templates, prompt templates.

> Let's standardise the communication between any two components.





### What is **Grammar**?
### Why do we need **Grammar** in language?

## Components

<!-- ![alt text](https://drive.google.com/uc?export=view&id=1YDf49ft1iV81tg1nWfTfFcbDZda9OfTm) -->


![alt text](https://drive.google.com/uc?export=view&id=1HPMN42P2w6Grn38mb5Rxre7OEVzK47oE)
<!--
<img src="https://drive.google.com/uc?export=view&id=1HPMN42P2w6Grn38mb5Rxre7OEVzK47oE" width="1500"> -->


1. Chains
2. Memory
3. Agents & Tools




## 2.1. Chains

![](https://drive.google.com/uc?export=view&id=13btaEa3XJQbV_YwRJgK4xJ5loeKGiSEK)

LangChain’s core abstraction is the **chain**:

> input → prompt → LLM → parser → output

In this section you will see:

- `ChatPromptTemplate` - how we define parameterised prompts like `"Tell me a joke about {topic}"`.
- `StrOutputParser` - a simple parser that turns the model's response into a plain Python string.
- **LCEL composition (`|`)** - how to connect prompt → model → parser into a single reusable object.
- How to **compose chains**: one chain writes a short story, another reviews it, and we plug them together.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 1. Create the Prompt
# It expects a dictionary like {"topic": "..."}
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}.")

# 2. Create the Parser
output_parser = StrOutputParser()

# 3. Build the Chain using LCEL
# The data flows from left to right
chain = prompt | llm | output_parser

# 4. Run the Chain
response = chain.invoke({"topic": "software engineers"})

print(response)

Why did the software engineer move to Pune?

...Because they heard the traffic was a great debugging challenge! 

(Pune is known for its notoriously challenging traffic!)


In [ ]:
# Chain 1: The Comedian
prompt1 = ChatPromptTemplate.from_template("Tell me a short joke about {topic}.")
chain1 = prompt1 | llm | StrOutputParser()

# Chain 2: The Audience
# Note: This prompt expects the input to be the joke text itself
prompt2 = ChatPromptTemplate.from_template("Write a sarcastic reply on did you find this funny: {joke}")
chain2 = prompt2 | llm | StrOutputParser()

In [ ]:
respons1 = chain1.invoke({"topic": "teddy bear"})
respons2 = chain2.invoke({"joke": respons1})

In [ ]:
respons1

'Why did the teddy bear say no to dessert? \n\n... Because she was stuffed! \n\n😂'

In [ ]:
respons2

"Oh, *hilarious*. Truly groundbreaking comedy. I haven't laughed that hard since... well, since the last time someone told that joke. It's so original, so unexpected. My sides are *aching* from the sheer wit. 😂 (Please send help, I think I'm developing a permanent eye-roll.) \n\nSeriously though, it's a classic. A *very* classic. You've really outdone yourself. 🙄"

In [ ]:
# The 'RunnablePassthrough' or a simple dictionary map helps us bridge the two.
# We map the output of chain1 to the input key "story" for chain2.
overall_chain = {"joke": chain1} | chain2

# Run it!
print(overall_chain.invoke({"topic": "teddy bear"}))

Oh. My. God. Absolutely *riveting*. I haven't laughed this hard since... well, probably since the last time someone told me a teddy bear joke. Truly, a comedic masterpiece. I'm going to need a moment to recover from the sheer brilliance. 😂 (Is that the level of sarcasm you were hoping for?)


## 2.2 Memory

![](https://drive.google.com/uc?export=view&id=1I2bPOe47vmLiUfyQEVaMXXNXrY0hBQH1)
By default, chains are **stateless**: every call forgets previous turns.

To build a real chat experience we need **memory**, i.e., a way to keep track of past messages and feed them back to the model. In this example we use:

- `ChatMessageHistory` - an in-memory list of messages for each conversation.
- `MessagesPlaceholder("history")` - a slot in the prompt where we inject the past messages.
- `RunnableWithMessageHistory` - a wrapper that automatically:
  - reads the right history for a given `session_id`
  - appends new user/assistant messages after each call.

After running the example, the model correctly answers *“What did I just tell you about my work?”* by reading from the stored history.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

# 1. Define a chat prompt with a history placeholder
chat_prompt = ChatPromptTemplate.from_messages([
    ("human", "You are a friendly assistant that remembers details about the user."),
    MessagesPlaceholder("history"),
    ("human", "{input}"),
])

chat_chain = chat_prompt | llm | StrOutputParser()

# 2. In-memory store for multiple sessions
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 3. Wrap chain with RunnableWithMessageHistory
chat_chain_with_memory = RunnableWithMessageHistory(
    chat_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)




In [ ]:

session_id = "demo-session"

for user_input in [
    "Hi, I'm Chinmay and I work with LangChain.",
    "What did I just tell you about my work?",
]:
    response = chat_chain.invoke(
        {"input": user_input, "history": [""]},
        config={"configurable": {"session_id": session_id}},
    )
    print(f"User: {user_input}")
    print(f"Assistant: {response}")
    print("-" * 50)

# 4. Demo conversation
session_id = "demo-session"

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:2745: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


User: Hi, I'm Chinmay and I work with LangChain.
Assistant: Hi Chinmay! It's great to meet you. So you're Chinmay, and you work with LangChain - that's fantastic! I'll remember that. 

LangChain is really interesting stuff - are you building anything cool with it at the moment? I'm always eager to hear what people are doing with these kinds of tools.

Just let me know if I can help you with anything at all. 😊
--------------------------------------------------
User: What did I just tell you about my work?
Assistant: You just told me you're a software engineer working on a project involving a large language model! Specifically, you mentioned you're working on making it more reliable and less prone to "hallucinations" - making things up. 

It sounds like interesting work! 😊 Is there anything I can help you with regarding that, or anything else?
--------------------------------------------------


In [ ]:

for user_input in [
    "Hi, I'm Chinmay and I work with LangChain.",
    "What did I just tell you about my work?",
]:
    response = chat_chain_with_memory.invoke(
        {"input": user_input},
        config={"configurable": {"session_id": session_id}},
    )
    print(f"User: {user_input}")
    print(f"Assistant: {response}")
    print("-" * 50)

User: Hi, I'm Chinmay and I work with LangChain.
Assistant: Hi Chinmay! It's great to meet you. So you're Chinmay, and you work with LangChain - that's fantastic! I'll remember that. 

LangChain is really interesting stuff - are you building anything cool with it at the moment? I'm always eager to hear what people are doing with these kinds of tools.

Just let me know if I can help you with anything at all. 😊
--------------------------------------------------
User: What did I just tell you about my work?
Assistant: You just told me that you're Chinmay and you work with LangChain! I'm doing my best to remember things - glad to see I got that one right. 😊 

Is there anything specific about your work with LangChain you'd like to talk about?
--------------------------------------------------


## 2.3 Agents and Tools
![](https://drive.google.com/uc?export=view&id=1Ppm_f4xRHIjnTexrHoY0_98zCU8CIFXg)

### 2.3.1 Adding Tools

Tools wrap **external capabilities** and present them to the LLM as callable functions.

Here we define two tools:

- `DuckDuckGoSearchRun` - for general web search.
- `WikipediaQueryRun` - for quick encyclopaedic lookups.

We then:

1. Put them into a `tools` list.
2. Call `search.run(...)` directly to see what a tool returns.
3. Reuse the same `tools` list later when we build agents and the guessing game.

The key idea: tools are your bridge from *LLM thinking* to *real-world actions* (APIs, DB calls, code, etc.).

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# 1. Define the Search Tool
search = DuckDuckGoSearchRun()

# 2. Define the Wikipedia Tool
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# 3. Create a "Toolbelt" (List of tools)
tools = [search, wikipedia]

# Test the tools individually to see what they return
print("Search Test:", search.run("Current stock price of Google"))

Search Test: Get detailed information about the Alphabet Inc. stock (GOOGL) including Price , Charts, Technical Analysis, Historical data, Alphabet Reports and more. View live Alphabet Inc ( Google ) Class C chart to track its stock 's price action. Find market predictions, GOOG financials and market news.The current price of GOOG is 322.09 USD — it has increased by 1.16% in the past 24 hours. Find the latest Alphabet Inc. (GOOG) stock quote, history, news and other vital information to help you with your stock trading and investing.Buy. Analyst Price Targets. 185.00 Low. 320.43 Average. 322.09 Current . The stock currently trades at a forward price -to-earnings ratio of 18 times based on analysts' estimates for 2025. Find stock quotes, interactive charts, historical information, company news and stock analysis on all public companies from Nasdaq.


### 2.3.2 Add Agents

Agents are **LLM‑powered controllers**.

Given:

- a model (`llm`), and
- a list of tools (`tools`),

an agent will decide **which tool to call, with what arguments, and when to stop**. Internally, the loop looks like:

1. Think → decide what to do.
2. Act → call a tool.
3. Observe → read the result.
4. Repeat until it has an answer.

In this section we use `create_agent` to hide that loop behind a single convenient interface, then let the agent answer a question using the search + Wikipedia tools you defined earlier.

In [ ]:
llm_gemini = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite", # Changed from gemma-3-27b-it to a model that supports function calling
    temperature=0.7,
)

In [ ]:
from langchain.agents import create_agent

# 1. Create the Agent
# This "all-in-one" function creates a compiled agent graph.
# It handles the loop (Thought -> Action -> Observation) automatically.
agent = create_agent(
    model=llm_gemini,
    tools=tools,
    system_prompt="You are a helpful assistant. Use your tools to answer questions."
)

# 2. Run the Agent
# Note: We pass 'messages' because this is a Chat Agent.
response = agent.invoke({
    "messages": [("human", "Who is the current CEO of Google and how old is he?")]
})

# 3. Print the Result
# The response is a dictionary. The final answer is usually the last message.
print(response["messages"][-1].content)

In [ ]:
from langchain_core.tools import tool

@tool
def get_project_info(project_code: str) -> str:
    '''
    Gets project info from the database.
    '''
    return f"Info about project {project_code}"

In [ ]:
tools.append(get_project_info)

In [ ]:
# help(agent)

## 2.4 More on memory
![](https://drive.google.com/uc?export=view&id=1Tbiq_2bAmbLaK5wl_00hnFbpC_4z1mPa)

## 2.5 Langchain Ecosystem

![alt text](https://drive.google.com/uc?export=view&id=1M9Q3f1oGEWy09tlTLLi4QITJeg7rDCEj)

## 3. Mini Project - Guess Who Agent with Tools (Notebook UI)

In this game:

- **You** secretly think of a famous real or fictional person.
- The **agent** asks you questions (mostly yes/no) to figure out who it is.
- The agent can optionally use **web search and Wikipedia tools** internally to check facts.

We also add a tiny notebook UI using `ipywidgets` so you can play without writing `input()` loops.

### 3.1 Create the Guess Who agent (with web tools)

We reuse the `search` and `wikipedia` tools defined earlier and give them to a new agent.
The agent follows a simple protocol:

- If it's asking a question: start the message with `QUESTION:`  
- If it's making a guess: start the message with `GUESS:`

In [ ]:
# 2. Define a callback that sleeps after each LLM call
from langchain_core.callbacks import BaseCallbackHandler

class SleepAfterLLMHandler(BaseCallbackHandler):
    def __init__(self, delay: float = 2.0):
        self.delay = delay

    def on_llm_end(self, *args, **kwargs):
        # This is triggered after EVERY LLM call inside the agent
        time.sleep(self.delay)

In [ ]:
# --- Guess Who agent that can use web tools ---

guess_who_tool_agent = create_agent(
    model=llm_gemini,
    tools=tools,  # reuses [search, wikipedia] from above
    system_prompt=(
        "You are playing a game called 'Guess Who' which is a game like Akinator. "
        "The user is secretly thinking of a well-known real or fictional person.\n\n"
        "Your goal is to identify the person by asking a sequence of questions.\n"
        "Also try to ask smart questions as you have to keep the number of questions minimum.\n"
        "- Ask one concise question at a time.\n"
        "- Prefer YES/NO questions, but short open questions are okay if needed.\n"
        "- After you think you know the answer, make a guess.\n\n"
        "You have access to web search and Wikipedia tools to look up facts about people "
        "(for example, to check if a candidate matches the clues you have).\n\n"
        "Output protocol:\n"
        "- If you are asking a question, start with 'QUESTION: '.\n"
        "- If you are making a guess, start with 'GUESS: '.\n"
        "Do not show tool call details to the user; use them internally."
    ),
)

print("Guess Who tool-using agent is ready!")

Guess Who tool-using agent is ready!


### 3.2 Simple notebook UI with `ipywidgets`

Use the buttons + text box to:

1. Click **Start New Game** - the agent will ask the first question.
2. Type your answer in the text box and click **Send Answer** each turn.
3. When the agent prints `GUESS: ...`, you can decide if it was correct and either keep playing or start a new game.

> Tip: Think of a person *before* you click **Start New Game** 🙂

In [ ]:
import time
import ipywidgets as widgets
from IPython.display import display, clear_output

# Simple UI for playing Guess Who inside the notebook

# Widgets
start_button = widgets.Button(description="Start New Game", button_style="success")
answer_box = widgets.Text(placeholder="Type your answer here (yes/no/short text)...")
send_button = widgets.Button(description="Send Answer", button_style="primary")
output_area = widgets.Output()

# Conversation state (messages list used by the agent)
guess_messages = []

def start_game(_):
    global guess_messages
    guess_messages = [("human", "START")]
    with output_area:
        clear_output()
        print("Think of a famous real or fictional person, then answer the questions.")
        print("Agent is thinking (20 seconds)...")
        time.sleep(20)  # 20-second delay before first request
        response = guess_who_tool_agent.invoke({"messages": guess_messages})
        guess_messages = response["messages"]
        print("Agent:", guess_messages[-1].content)

def send_answer(_):
    global guess_messages
    user_text = answer_box.value.strip()
    if not user_text:
        return
    answer_box.value = ""

    guess_messages.append(("human", user_text))

    with output_area:
        print("\nYou:", user_text)
        print("⏳ Agent is thinking (20 seconds)...")
        # time.sleep(20)  # 20-second delay before each follow-up request
        response = guess_who_tool_agent.invoke(
            {
                "messages": guess_messages
            },
            config={
                "callbacks": [SleepAfterLLMHandler(delay=20)]  # 20-second pause after EACH LLM call
            },
        )
        guess_messages = response["messages"]
        print("Agent:", guess_messages[-1].content)

start_button.on_click(start_game)
send_button.on_click(send_answer)

ui = widgets.VBox([
    start_button,
    widgets.HBox([answer_box, send_button]),
    output_area,
])

display(ui)


## UPCOMING - RAG

In this section we build a tiny **retrieval-augmented generation (RAG)** pipeline:

1. **Load** documents from a folder with `DirectoryLoader` + `PyPDFLoader`.
2. **Split** them into overlapping text chunks with `RecursiveCharacterTextSplitter`.
3. **Embed & index** the chunks in a FAISS vector store using a BGE embedding model.
4. **Ask questions**: a RAG chain retrieves relevant chunks and lets Gemini answer *based only on those documents*.

This is the “serious” mini-project, compared to the game: it shows how the same LangChain building blocks
(prompts, chains, retrievers) can power a real-world document-QA workflow.